# Bayes ingenuo Tarea 1
### Equipo 11
- Iñaki Fernandez Fiscal
- Guillermo Arredondo Renero
- Mauricio Vazquez Moran

Este programa clasifica correos electrónicos como spam o ham utilizando el alrogítmo de bayes ingenuo.

In [1]:
usePackage <- function(p) 
{
  if (!is.element(p, installed.packages()[,1]))
    install.packages(p, repos = "")
  suppressPackageStartupMessages(require(p, character.only = TRUE, quietly  = TRUE))
}

In [2]:
usePackage('R.utils')
usePackage('tm')

Warning message:
"package 'tm' was built under R version 3.6.3"Warning message:
"package 'NLP' was built under R version 3.6.3"

## Descarga los datos 

In [3]:
download.mails <- function(url, dir_name, file_name){

  if (!file.exists(dir_name)) {
    dir.create(dir_name)  
  }
  
  download.file(url, destfile=file.path(dir_name, paste0(file_name,".tar.bz2")) )
  bunzip2(file.path(dir_name, paste0(file_name,".tar.bz2")))
  
  untar(file.path(dir_name, paste0(file_name,".tar")), exdir = dir_name)
  
  if (file.exists(file.path(dir_name, paste0(file_name,".tar")))) {
    file.remove(file.path(dir_name, paste0(file_name,".tar")))
  }
  
}

In [4]:
dir_name <- "data"
file_name <- "easy_ham_2"
url <- "http://spamassassin.apache.org/old/publiccorpus/20030228_easy_ham_2.tar.bz2"

download.mails(url, dir_name, file_name)

[1] TRUE

In [5]:
url <- "http://spamassassin.apache.org/old/publiccorpus/20030228_hard_ham.tar.bz2"
file_name <- "hard_ham"

download.mails(url, dir_name, file_name)

[1] TRUE

In [6]:
url <- "http://spamassassin.apache.org/old/publiccorpus/20030228_spam_2.tar.bz2"
file_name <- "spam_2"

download.mails(url, dir_name, file_name)

[1] TRUE

## Preprocesamiento de los correos electrónicos

In [7]:
# Hacemos una función que leea el mensaje del archivo que se le pase como parámetro
# asumimos que el archivo contiene un correo

lee_mensaje <- function(correo) {
  fd <- file(correo, open = "rt") # regresa un lector de archivo
  lineas <- readLines(fd, warn=FALSE) # lee linea por linea y genera un vector de cadenas de caracteres
  close(fd)
  mensaje <- lineas[seq(which(lineas == "")[1] + 1, length(lineas), 1)] # which trae indices del vector que cumplen condicion
  return (paste(mensaje, collapse = "\n")) # paste concatena sinser el primer caracter, collapse evita que sea vector y sea todo una cadena
}

In [8]:
# Creamos variables con los directorios donde se encuentran los datos
trayectoria_spam     <- file.path(dir_name, "spam_2")
trayectoria_easyham  <- file.path(dir_name, "easy_ham_2")
trayectoria_hardham  <- file.path(dir_name, "hard_ham")

### Spam

In [9]:
# Leemos el directorio donde se encuentran los correos clasificados como spam
archivos_correos_spam <- dir(trayectoria_spam)

# quitamos el guión llamado cmds
archivos_correos_spam <- archivos_correos_spam[which(archivos_correos_spam!="cmds")] #[1:250]

# Aquí empieza la tarea
# Dividir entre entrenamiento y prueba, muestras aleatorias de los archivos, unos para entrenar y otros para probarlo
archivos_correos_spam <- archivos_correos_spam[sample(1:length(archivos_correos_spam))]
archivos_correos_spam_training <- archivos_correos_spam[1:200]
archivos_correos_spam_testing <- archivos_correos_spam[201:250]

# cambiar a training_spam
todo_spam <- sapply(archivos_correos_spam_training,
                   function(p) lee_mensaje(file.path(trayectoria_spam, p)))
                    
todo_spam <- enc2utf8(todo_spam)

### Easy ham

In [10]:
# Leemos el directorio donde se encuentran los correos clasificados como ham fácilmente identificables
archivos_correos_easy_ham <- dir(trayectoria_easyham)

# quitamos el guión llamado cmds
archivos_correos_easy_ham <- archivos_correos_easy_ham[which(archivos_correos_easy_ham!="cmds")] #[1:250]

archivos_correos_easy_ham <- archivos_correos_easy_ham[sample(1:length(archivos_correos_easy_ham))]
archivos_correos_easy_ham_training <- archivos_correos_easy_ham[1:200]
archivos_correos_easy_ham_testing <- archivos_correos_easy_ham[201:250]

todo_easy_ham <- sapply(archivos_correos_easy_ham_training,
                    function(p) lee_mensaje(file.path(trayectoria_easyham, p)))

todo_easy_ham <- enc2utf8(todo_easy_ham)

### Hard ham

In [11]:
# Leemos el directorio donde se encuentran los correos clasificados como ham fácilmente identificables
archivos_correos_hard_ham <- dir(trayectoria_hardham)

# quitamos el guión llamado cmds
archivos_correos_hard_ham <- archivos_correos_hard_ham[which(archivos_correos_hard_ham!="cmds")] #[1:250]

archivos_correos_hard_ham <- archivos_correos_hard_ham[sample(1:length(archivos_correos_hard_ham))]
archivos_correos_hard_ham_training <- archivos_correos_hard_ham[1:200]
archivos_correos_hard_ham_testing <- archivos_correos_hard_ham[201:length(archivos_correos_hard_ham)]

todo_hard_ham <- sapply(archivos_correos_hard_ham_training,
                    function(p) lee_mensaje(file.path(trayectoria_hardham, p)))

todo_hard_ham <- enc2utf8(todo_hard_ham)

## Preparación de corpus y bolsa de palabras

In [12]:
obtiene_TermDocumentMatrix <- function (vector_correos) {
  control <- list(stopwords = TRUE,
                removePunctuation = TRUE,
                removeNumbers = TRUE,
                minDocFreq = 2)
  corpus <- Corpus(VectorSource(vector_correos))
  return(TermDocumentMatrix(corpus, control))
}

### Spam

In [13]:
spam_TDM <- obtiene_TermDocumentMatrix(todo_spam)

# Crea un data frame que provee el conjunto de caracteristicas de los datos de entrenamiento SPAM
matriz_spam <- as.matrix(spam_TDM)

conteos_spam <- rowSums(matriz_spam)
df_spam <- data.frame(cbind(names(conteos_spam),
                            as.numeric(conteos_spam)),
                      stringsAsFactors = FALSE)
names(df_spam) <- c("terminos", "frecuencia")
df_spam$frecuencia <- as.numeric(df_spam$frecuencia)
ocurrencias_spam <- sapply(1:nrow(matriz_spam),
                          function(i) # Obtiene la proporcion de documentos que contiene cada palabra
                          {
                            length(which(matriz_spam[i, ] > 0)) / ncol(matriz_spam)
                          })
densidad_spam <- df_spam$frecuencia/sum(df_spam$frecuencia,na.rm = TRUE) #na.rm (remove na)

df_spam <- transform(df_spam,
                     densidad = densidad_spam,
                     ocurrencias = ocurrencias_spam)

In [14]:
head(df_spam)

terminos,frecuencia,densidad,ocurrencias
acceptance,1,1.777651e-05,0.005
accounts,5,8.888257e-05,0.010
actual,4,7.110605e-05,0.015
africa,2,3.555303e-05,0.005
agencies,14,2.488712e-04,0.030
allowed,2,3.555303e-05,0.010


### Easy ham

In [15]:
easy_ham_TDM <- obtiene_TermDocumentMatrix(todo_easy_ham)

# Crea un data frame que provee el conjunto de caracteristicas de los datos de entrenamiento easy ham
matriz_easy_ham <- as.matrix(easy_ham_TDM)

conteos_easy_ham <- rowSums(matriz_easy_ham)
df_easy_ham <- data.frame(cbind(names(conteos_easy_ham),
                            as.numeric(conteos_easy_ham)),
                      stringsAsFactors = FALSE)
names(df_easy_ham) <- c("terminos", "frecuencia")
df_easy_ham$frecuencia <- as.numeric(df_easy_ham$frecuencia)
ocurrencias_easy_ham <- sapply(1:nrow(matriz_easy_ham),
                           function(i) # Obtiene la proporcion de documentos que contiene cada palabra
                           {
                             length(which(matriz_easy_ham[i, ] > 0)) / ncol(matriz_easy_ham)
                           })
densidad_easy_ham <- df_easy_ham$frecuencia/sum(df_easy_ham$frecuencia,na.rm = TRUE)

df_easy_ham <- transform(df_easy_ham,
                     densidad = densidad_easy_ham,
                     ocurrencias = ocurrencias_easy_ham)

In [16]:
head(df_easy_ham)

terminos,frecuencia,densidad,ocurrencias
administrator,3,1.029760e-04,0.015
august,17,5.835307e-04,0.060
beeping,1,3.432534e-05,0.005
brady,5,1.716267e-04,0.025
computing,3,1.029760e-04,0.015
department,7,2.402773e-04,0.025


### Hard ham

In [17]:
hard_ham_TDM <- obtiene_TermDocumentMatrix(todo_hard_ham)

# Crea un data frame que provee el conjunto de caracteristicas de los datos de entrenamiento easy ham
matriz_hard_ham <- as.matrix(hard_ham_TDM)

conteos_hard_ham <- rowSums(matriz_hard_ham)
df_hard_ham <- data.frame(cbind(names(conteos_hard_ham),
                            as.numeric(conteos_hard_ham)),
                      stringsAsFactors = FALSE)
names(df_hard_ham) <- c("terminos", "frecuencia")
df_hard_ham$frecuencia <- as.numeric(df_hard_ham$frecuencia)
ocurrencias_hard_ham <- sapply(1:nrow(matriz_hard_ham),
                           function(i) # Obtiene la proporcion de documentos que contiene cada palabra
                           {
                             length(which(matriz_hard_ham[i, ] > 0)) / ncol(matriz_hard_ham)
                           })
densidad_hard_ham <- df_hard_ham$frecuencia/sum(df_hard_ham$frecuencia,na.rm = TRUE)

df_hard_ham <- transform(df_hard_ham,
                     densidad = densidad_hard_ham,
                     ocurrencias = ocurrencias_hard_ham)

In [18]:
head(df_hard_ham)

terminos,frecuencia,densidad,ocurrencias
aac,4,1.706499e-05,0.015
ability,30,1.279874e-04,0.100
able,51,2.175786e-04,0.170
aboutcom,1,4.266248e-06,0.005
access,112,4.778198e-04,0.235
accessing,5,2.133124e-05,0.020


## Cálculo de probabilidad a posteriori

In [19]:
a_posteriori <- function(trayectoria, df_entrenamiento, a_priori = 0.5, c = 1e-6)
{
  mensaje <- lee_mensaje(trayectoria)
  mensaje <- enc2utf8(mensaje)
  mensaje_TDM <- obtiene_TermDocumentMatrix(mensaje)
  conteos_mensaje <- rowSums(as.matrix(mensaje_TDM))

  # Encuentra palabras en data frame de entrenamiento
  mensaje_palabras_comunes <- intersect(names(conteos_mensaje), df_entrenamiento$terminos)
  
  # Ahora sólo aplicamos la clasificación Bayes ingenuo
  if(length(mensaje_palabras_comunes) < 1)
  {
    #return(a_priori * c ^ (length(conteos_mensaje)))
    return(log(a_priori) + (length(conteos_mensaje)) *log(c))
  }
  else
  {
    probabilidades_palabras_comunes <- df_entrenamiento$densidad[match(mensaje_palabras_comunes, df_entrenamiento$terminos)]
    #return(a_priori * prod(probabilidades_palabras_comunes) * c ^ (length(conteos_mensaje) - length(mensaje_palabras_comunes)))
    return(log(a_priori) + sum(log(probabilidades_palabras_comunes)) + log(c) * (length(conteos_mensaje) - length(mensaje_palabras_comunes)))
  }
}

In [20]:
install.packages("ramify")

package 'ramify' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\memit\AppData\Local\Temp\RtmpuSvaSo\downloaded_packages


In [21]:
library(ramify)

Warning message:
"package 'ramify' was built under R version 3.6.3"
Attaching package: 'ramify'

The following object is masked from 'package:graphics':

    clip



## Clasificación (modificacion)

In [22]:
clasifica_spam_max <- function(trayectoria, archivos) {

  spam_prueba <- sapply(archivos,
                             function(p) a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_spam))
  ham_prueba <- sapply(archivos,
                            function(p) a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_easy_ham))
  hard_ham_prueba <- sapply(archivos,
                            function(p) a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_hard_ham))
                                
  x <- matrix(c(ham_prueba, hard_ham_prueba, spam_prueba), nrow=length(ham_prueba), ncol=3)
  return(as.factor(argmax(x)))
                        
}

### Clasificacion anterior

In [23]:
clasifica_spam <- function(trayectoria, archivos) {

  hard_ham_spam_prueba <- sapply(archivos,
                             function(p) a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_spam))
  hard_ham_ham_prueba <- sapply(archivos,
                            function(p) a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_hard_ham))
  
  return (ifelse(hard_ham_spam_prueba > hard_ham_ham_prueba,
                        TRUE,
                        FALSE))
}

### Resultados con 2 var (hard ham)

In [24]:
# Leemos el directorio donde se encuentran los correos clasificados como ham dificlmente identificables
archivos_correos_easy_ham_2 <- dir(trayectoria_easyham)

# quitamos el guión llamado cmds
archivos_correos_easy_ham_2 <- archivos_correos_easy_ham[which(archivos_correos_easy_ham!="cmds")]
archivos_correos_easy_ham_2 <- archivos_correos_easy_ham_2[sample(1:length(archivos_correos_easy_ham_2))]
archivos_correos_easy_ham_2 <- archivos_correos_easy_ham_2[201:250]

In [25]:
hard_ham_res <- clasifica_spam(trayectoria_hardham, archivos_correos_hard_ham_testing)
easy_ham_res <- clasifica_spam(trayectoria_easyham, archivos_correos_easy_ham_2)
spam_res     <- clasifica_spam(trayectoria_spam,    archivos_correos_spam_testing)

In [26]:
summary(easy_ham_res)
summary(spam_res)
summary(hard_ham_res)

   Mode   FALSE    TRUE 
logical      40      10 

   Mode   FALSE    TRUE 
logical       1      49 

   Mode   FALSE    TRUE 
logical      45       5 

### Entrenamiento modificado

In [27]:
easy_ham_res <- clasifica_spam_max(trayectoria_easyham, archivos_correos_easy_ham_testing)
hard_ham_res <- clasifica_spam_max(trayectoria_hardham, archivos_correos_hard_ham_testing)
spam_res     <- clasifica_spam_max(trayectoria_spam,    archivos_correos_spam_testing)

## Resultados

In [28]:
summary(easy_ham_res)
summary(hard_ham_res)
summary(spam_res)

1 
50

1  2  3 
 4 42  4

1  2  3 
 2  1 47

# Conclusiones Generales

1. **Entrenamiento con easy-ham y spam:** 
El modelo clasifica de manera perfecta el easy ham, pero se debe a que entrenamos con easy ham. Es muy bueno para calcular el easy ham debido a que son los datos que le presentamos al modelo. Lo correcto seria presentar una muestra estratificada o que tuviera tanto hard ham como easy ham. Con una muestra donde tuviera 80% de easy ham + 80% de hard ham haría que tuviera una mayor precisión. Sin embargo, no es tan malo para predecir Spam, pues tiene un ~5% de error para el de dataset de Spam, pero para hard ham tiene ~45% de error. Es casi lo equivalente a un volado...
2. **Entrenamiento con hard-ham y spam:**
Este entrenamiento resulta en una aparente mejora en comparación al entrenamiento con easy-ham, pues la eficiencia de hard-ham mejora, evidente y notablemente; sin embargo, es importante darnos cuenta que esto se debe a que ahora entrenamos con esa muestra, por lo que tiene muchísima más información con la cual comparar; lamentablemente esto resulta en un decaimiento esperado del rendimiento de easy-ham. No obstante, hay que notar que el porcentaje de error con la prueba de easy-ham es mucho mejor que en el caso 1, siendo ~70%. Además, detecta casi perfectamente a los spam (en algunos casos perfectamente); esto nos indica un mejor aprendizaje de los ham's en general en comparación al entrenamiento con easy-ham. 
3. **Comparación con 3 variables en matriz:**
Ahora observamos dos cosas importantes: en primer lugar, el ajuste general de entrenamiento resulta satisfactorio en proporciones mucho más significantes que en los dos casos anteriores, pues vemos que logra identificar con gran precisión los tres conjuntos de características. En segundo lugar, podemos observar una relación interesante en el hecho de que las dos clases, aparentemente más separadas (easy-ham y spam) las logra clasificar casi perfectamente ( ~94%-98% ), al mismo tiempo que en el caso de la variable que intersecta entre ambas (hard-ham), la precisión de detección de spam se mantiene estable ( ~92%-94% ).
Por último, consideramos que podríamos verificar mejor el modelo al dar muestras que se acerquen más al 80-20 para cada variable. 